Needed imports and utility

In [1]:
import os
import ansys.speos.script as script
import ansys.speos.core as core

tests_data_path = os.path.join(os.path.join(os.path.abspath(''), os.path.pardir), "assets")

def clean_all_dbs(speos_client: core.SpeosClient):
    for item in (
        speos_client.jobs().list()
        + speos_client.scenes().list()
        + speos_client.simulation_templates().list()
        + speos_client.sensor_templates().list()
        + speos_client.source_templates().list()
        + speos_client.intensity_templates().list()
        + speos_client.spectrums().list()
        + speos_client.vop_templates().list()
        + speos_client.sop_templates().list()
        + speos_client.parts().list()
        + speos_client.bodies().list()
        + speos_client.faces().list()
    ):
        item.delete()

tests_data_path = os.path.join(os.path.join(os.path.abspath(''), os.path.pardir), "assets")

Create connection with speos rpc server

In [2]:
speos = core.Speos(host="localhost", port=50098)
clean_all_dbs(speos.client)

New project

In [ ]:
p = script.Project(speos=speos)
print(p)

# PreRequisites

At least a sensor is needed. For example we create two sensors and a source

In [ ]:
sensor1 = p.create_sensor(name="sensor.1")
sensor1.set_radiance()
sensor1.commit()

sensor2 = p.create_sensor(name="sensor.2")
sensor2.set_radiance()
sensor2.commit()

source1 = p.create_source(name="source.1")
source1.set_surface()
source1.commit()

# Create Simulation
Create locally  
The mention "local: " is added when printing the simulation

In [ ]:
simulation1 = p.create_simulation(name="Simulation.1")
print(simulation1)

# Simulation Setup 

In [ ]:
simulation1.set_sensor_paths(sensor_paths=["sensor.1", "sensor.2"])
simulation1.set_source_paths(source_paths=["source.1"])

simulation1.set_geometries(values=[script.GeoRef.from_native_link(geopath="mybody1")\
                                       , script.GeoRef.from_native_link(geopath="mybody2")\
                                        , script.GeoRef.from_native_link(geopath="mybody3")])
print(simulation1)

## Simulation Direct

To set up a direct Monte Carlo (MC) simulation, we first call the `set_direct_MC_simulation` method on the `simulation1` object.    
Finally, we print the `simulation1_direct` object to verify the setup.

In [ ]:
simulation1.set_direct()
print(simulation1)
print(p)

Push it to the server.  
Now that it is committed to the server, the mention "local: " is no more present when printing the simulation1.

In [ ]:
print(p)

In [ ]:
simulation1.commit()
print(simulation1)

Another example by setting several characteristics

In [ ]:
material_file_path = os.path.join(tests_data_path, "material.material")
simulation2 = p.create_simulation(name="Simulation.2")

simulation2_direct = simulation2.set_direct()
simulation2_direct  .set_ambient_material_file_uri(url=material_file_path)\
                    .set_colorimetric_standard_CIE_1964()\
                    .set_weight_none()\
                    .set_geom_distance_tolerance(0.01)\
                    .set_max_impact(200)\
                    .set_fast_transmission_gathering()\
                    .set_dispersion(False)
simulation2.commit()
print(simulation2)

## Default values
Some default values are available when applicable in every methods and class.

# Read
## Simulation Instance
A mention "local: " is added if it is not yet committed to the server

In [ ]:
print(simulation1)

# Project

In [ ]:
print(p)

# Update
Tipp: if you are manipulating a simulation already committed, don't forget to commit your changes.  
If you don't, you will still only watch what is committed on the server.

In [ ]:
simulation1.set_sensor_paths(sensor_paths=["sensor.1", "sensor.2"])
simulation1.set_source_paths(source_paths=["source.1"])

simulation1.set_geometries(values=[script.GeoRef.from_native_link(geopath="mybody1")\
                                       , script.GeoRef.from_native_link(geopath="mybody2")\
                                        , script.GeoRef.from_native_link(geopath="mybody3")])

simulation1_direct = simulation1.set_direct()
simulation1_direct  .set_ambient_material_file_uri(url=material_file_path)\
                    .set_colorimetric_standard_CIE_1964()\
                    .set_weight_none()\
                    .set_geom_distance_tolerance(0.01)\
                    .set_max_impact(200)\
                    .set_fast_transmission_gathering()\
                    .set_dispersion(False)
simulation1.commit()

## Reset
Possibility to reset local values from the one available in the server.

In [ ]:
simulation1.set_direct() # set direct but no commit
simulation1.reset() # reset -> this will apply the server value to the local value
simulation1.delete() # delete (to display the local value with the below print)
print(simulation1)

# Delete
Once the data is deleted from the server, you can still work with local data and maybe commit later.

In [ ]:
simulation2.delete()
print(simulation2)

In [ ]:
simulation1.delete()

# More content
## Inverse Simulation 

In [ ]:
material_file_path = os.path.join(tests_data_path, "material.material")
simulation3 = p.create_simulation(name="Simulation.3")

simulation3_inverse = simulation3.set_inverse()
simulation3_inverse  .set_ambient_material_file_uri(url=material_file_path)\
                    .set_colorimetric_standard_CIE_1964()\
                    .set_weight_none()\
                    .set_geom_distance_tolerance(0.01)\
                    .set_max_impact(200)\
                    .set_splitting(True)\
                    .set_number_of_gathering_rays_per_source(2)\
                    .set_maximum_gathering_error(3)\
                    .set_fast_transmission_gathering(True)\
                    .set_dispersion(False)

simulation3.commit()
print(simulation3)

## Interactive

In [ ]:
material_file_path = os.path.join(tests_data_path, "material.material")
simulation4 = p.create_simulation(name="Simulation.4")

simulation4_inverse = simulation4.set_interactive()
simulation4_inverse  .set_ambient_material_file_uri(url=material_file_path)\
                    .set_colorimetric_standard_CIE_1964()\
                    .set_weight_none()\
                    .set_geom_distance_tolerance(0.01)\
                    .set_max_impact(200)\

simulation4.commit()
print(simulation4)

# Cleanup

In [ ]:
clean_all_dbs(speos.client)